# Proyecto Machine Learning

## Instaladores

In [6]:
#%pip install ultralytics
#%pip install opencv-python
#%pip install roboflow
#%pip install bdd100k

## Bibliotecas

In [ ]:
# Herramientas de tratamiento de datos
import numpy as np

# Herramienimport ultralytics
from ultralytics import YOLO
from roboflow import Roboflowtas de computer vision
import cv2


# Herramientas de manejo de archivos
import os
import shutil
import zipfile
import requests
from io import BytesIO

## Conexión a Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ruta_drive = "/content/drive/MyDrive/ML_The_Bridge"

In [ ]:
rf = Roboflow(api_key="hCNbQPHCydCak3y2BI1B")

## Carga de YOLOv8m

In [3]:
modelo_YOLO = YOLO('yolov8n.pt')

## Carga de datasets

### COCO

Las imágenes y etiquetas de validación se descargan de forma automática al entrenar el modelo.  
Las imágenes de train se descargan del enlace proporcionado en el .yaml que del repo de GitHub de Ultralytics.
Las etiquetas de train se descargan con el siguiente comando:

In [8]:
from ultralytics.data.converter import convert_coco

convert_coco(
    json_path=r"C:\Users\adria\Bootcamp\ML_The_Bridge\ML_The_Bridge_dataset\coco_carpeta_temporal\annotations_trainval2017\annotations\instances_train2017.json",
    save_dir=r"C:\Users\adria\Bootcamp\ML_The_Bridge\ML_The_Bridge_dataset\coco_carpeta_temporal\train",
    use_keypoints=False,
)

TypeError: convert_coco() got an unexpected keyword argument 'json_path'

In [5]:
from ultralytics.data.converter import convert_coco
import os

# 1. Definir la RAÍZ donde se encuentra la estructura COCO (annotations/ e images/)
COCO_ROOT = r"C:\Users\adria\Bootcamp\ML_The_Bridge\ML_The_Bridge_dataset\coco" 
# ^ Asegúrate de que esta carpeta contenga las subcarpetas 'annotations' y 'images'.

# 2. Definir la CARPETA DE SALIDA (será la carpeta superior que contendrá 'labels/train2017')
OUTPUT_BASE_DIR = r"C:\Users\adria\Bootcamp\ML_The_Bridge\ML_The_Bridge_dataset\coco\labels"
os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)

# Rutas de los archivos JSON de entrada
TRAIN_JSON = os.path.join(COCO_ROOT, "annotations", "instances_train2017.json")
VAL_JSON = os.path.join(COCO_ROOT, "annotations", "instances_val2017.json")

# --- CONVERSIÓN DEL CONJUNTO DE ENTRENAMIENTO ---
print("Convirtiendo JSON de entrenamiento a YOLO...")
convert_coco(
    # labels_dir apunta al archivo JSON
    labels_dir=TRAIN_JSON,
    # save_dir apunta a la raíz de la salida. Creará: OUTPUT_BASE_DIR/labels/train2017
    save_dir=OUTPUT_BASE_DIR 
)
print("✅ Conversión de entrenamiento completada.")

# --- CONVERSIÓN DEL CONJUNTO DE VALIDACIÓN ---
print("Convirtiendo JSON de validación a YOLO...")
convert_coco(
    labels_dir=VAL_JSON,
    save_dir=OUTPUT_BASE_DIR
)
print("✅ Conversión de validación completada.")

Convirtiendo JSON de entrenamiento a YOLO...
COCO data converted successfully.
Results saved to C:\Users\adria\Bootcamp\ML_The_Bridge\ML_The_Bridge_dataset\coco\labels2
✅ Conversión de entrenamiento completada.
Convirtiendo JSON de validación a YOLO...
COCO data converted successfully.
Results saved to C:\Users\adria\Bootcamp\ML_The_Bridge\ML_The_Bridge_dataset\coco\labels3
✅ Conversión de validación completada.


### Señales de tráfico europeas

In [ ]:
project = rf.workspace("radu-oprea-r4xnm").project("traffic-signs-detection-europe")
version = project.version(11)
dataset = version.download("yolov8")

!cp -r "{dataset.location}" "/content/drive/MyDrive/ML_The_Bridge/datasets/"

### Semáforos

In [ ]:
project = rf.workspace("student-esos5").project("traffic-light-v9orl")
version = project.version(3)
dataset = version.download("yolov8")

!cp -r "{dataset.location}" "/content/drive/MyDrive/ML_The_Bridge/datasets/"

## Preparación de las carpetas y el .yaml

In [ ]:

SEMAFOROS_DIR_IN = "labels_originales/semaforos/train"
SEÑALES_DIR_IN = "labels_originales/senales/train"
COCO_DIR_IN = "labels_originales/coco_parcial/train" # Asume que ya descargaste y convertiste a YOLO

# Directorio de salida (donde guardarás las etiquetas modificadas)
OUTPUT_DIR = "dataset_combinado/labels/train"

# Asegúrate de que el directorio de salida exista
os.makedirs(OUTPUT_DIR, exist_ok=True)


# --- REGLAS DE MAPEO ---

# 1. Semáforos (Original: 0, 1, 2 -> Nuevo: 12, 13, 14)
SEMAFOROS_MAP = {'0': '12', '1': '13', '2': '14'}

# 2. Señales (Original: 0-54 -> Nuevo: 15-69)
SEÑALES_OFFSET = 15

# 3. COCO (Necesitas el mapeo exacto de las clases COCO que estás usando)
# Original COCO ID -> Nuevo ID
COCO_MAP = {
    '0': '0',   # person
    '1': '1',   # bicycle
    '2': '2',   # car
    '3': '3',   # motorcycle
    '5': '4',   # bus
    '7': '5',   # truck
    '10': '6',  # fire hydrant
    '16': '7',  # dog
    '39': '8',  # bottle
    '56': '9',  # chair
    '63': '10',  # laptop
    '67': '11',  # fcell phone
}


def reindex_labels(input_dir, mapping_or_offset, is_offset=False):
    """Procesa los archivos de etiquetas y aplica el mapeo o el offset."""
    for filename in os.listdir(input_dir):
        if not filename.endswith('.txt'):
            continue
        
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(OUTPUT_DIR, filename)
        
        with open(input_path, 'r') as f:
            lines = f.readlines()
        
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if not parts: continue
            
            original_class = parts[0]
            
            if is_offset:
                # Caso Señales: aplicar un desplazamiento
                try:
                    new_class = str(int(original_class) + mapping_or_offset)
                except ValueError:
                    print(f"Error en el formato de clase en {filename}: {original_class}")
                    continue
            else:
                # Caso Semáforos/COCO: aplicar un mapeo directo
                if original_class not in mapping_or_offset:
                    print(f"Advertencia: Clase {original_class} no encontrada en el mapeo en {filename}")
                    continue
                new_class = mapping_or_offset[original_class]

            # Reemplazar la clase y mantener el resto de la línea
            new_lines.append(new_class + " " + " ".join(parts[1:]) + "\n")
            
        with open(output_path, 'w') as f:
            f.writelines(new_lines)
    print(f"Etiquetas de {input_dir} re-indexadas y guardadas en {OUTPUT_DIR}.")

#### COCO

In [ ]:
reindex_labels(f"{ruta_drive}/datasets/COCO/labels/val2017", COCO_MAP)

#### Semáforos

In [ ]:
# 1. Re-indexar semáforos
reindex_labels(SEMAFOROS_DIR_IN, SEMAFOROS_MAP)

#### Señales de tráfico

In [ ]:
# 2. Re-indexar señales
reindex_labels(SEÑALES_DIR_IN, SEÑALES_OFFSET, is_offset=True)

## Entrenamiento del modelo

## Enlaces

Señales de tráfico europeas:    https://universe.roboflow.com/radu-oprea-r4xnm/traffic-signs-detection-europe  
Semáforos:                      https://universe.roboflow.com/student-esos5/traffic-light-v9orl   

# Descarto este notebook

No he conseguido descargar las etiquetas de COCO para el train, por lo que no puedo entrenar un modelo combinado.  
Paso a otro enfoque usando únicamente COCO.